1. 1 слой 26 нейронов (сначала передать туда матрицу декодера, потом инициализировать рандомно) 
2. 2 слоя по 31 нейрона
3. 

1. бинарный вход перевести в вещественный в [0,1]


In [0]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive
# !pip install -q keras
# !pip install -q pandas
# !pip install -q numpy

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
import os 
import sys
import pandas as pd
import numpy as np

from sklearn import model_selection
from __future__ import absolute_import

os.chdir("/content/drive/Hamming")
sys.path.append("Hamming")
# import functions as f
# import model_lib 

In [4]:
!ls
import tensorflow as tf
tf.test.gpu_device_name()

best_model.hd5	drive	      hamming-codes.ipynb  README.md
dataset_files	functions.py  model_lib.py	   visualisation.ipynb


'/device:GPU:0'

In [0]:
# !git pull

### hamming_small.txt - маленький датасет
### hamming.txt - большой датасет

In [0]:
TRAIN_PATH = './dataset_files/hamming_small.txt'  # "/content/drive/Hamming/dataset_files/hamming.txt"

COLUMN_NAMES = ['plainword', 'codeword', 
                'id_error', 'bin_error', 'defective_codeword']

In [0]:
def hamming_distance(first: str, second: str) -> int:
    return len([1 for (x, y) in zip(first, second) if x != y])

In [0]:
def load_data():
    return pd.read_csv(TRAIN_PATH, sep=';', names=COLUMN_NAMES)

In [0]:
# def balance_classes():
#     '''
#     Датасет копируется в no_err, 
#     значение маски ошибки заменяется на нулевую маску, 
#     значение "испорченного" кодового слова заменяется на кодовое,
#     no_err добавляется к датасету в конце.
#     '''
    
#     no_err = data.copy()  # deep=True)
#     no_err['id_error'] = -1
#     no_err['bin_error'] = '0' * 31
#     no_err['defective_codeword'] = no_err['codeword']
#     return data.append(no_err, ignore_index=True)  # ignore_index для сквозной нумеракции итогового датасета


In [0]:
def make_features():
#     data['dec_defective_codeword'] = data['defective_codeword'][:].apply(lambda x: int(x, 2))

    for j in range(len(data['codeword'][0])):
        data['cod_' + str(j)] = data['codeword'][:].apply(lambda x: int(x[j]))

    for j in range(len(data['defective_codeword'][0])):
        data['def_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))

#     for j in range(len(data['bin_error'][0])):
#         data['mask_' + str(j)] = data['bin_error'][:].apply(lambda x: int(x[j]))
        
    for j in range(len(data['plainword'][0])):
        data['pln_' + str(j)] = data['plainword'][:].apply(lambda x: int(x[j]))
#     return data

In [0]:
def split_data(test_size): 
  train_data, test_data, train_labels, test_labels = \
    model_selection.train_test_split(data.loc[:, 'def_0':'def_30'], 
      data.loc[:, 'cod_0':'cod_30'], # 'mask_0':'pln_25' 
      test_size = test_size) 
  return np.array(train_data), np.array(test_data), np.array(train_labels), np.array(test_labels)

In [11]:
data = load_data()
print(data.shape)

(65536, 5)


In [0]:
# data = balance_classes()
# print(data.shape)

In [13]:
%%time
make_features()
print(data.shape)

(65536, 93)
CPU times: user 3.3 s, sys: 15 ms, total: 3.32 s
Wall time: 3.32 s


In [14]:
data.head()

,plainword,codeword,id_error,bin_error,defective_codeword,cod_0,cod_1,cod_2,cod_3,cod_4,...,pln_16,pln_17,pln_18,pln_19,pln_20,pln_21,pln_22,pln_23,pln_24,pln_25
0,01000001001110110101011001,0011101000001001110110101011001,-1,0000000000000000000000000000000,0011101000001001110110101011001,0,0,1,1,1,...,0,1,0,1,0,1,1,0,0,1
1,01000001001110110101011001,0011101000001001110110101011001,0,1000000000000000000000000000000,1011101000001001110110101011001,0,0,1,1,1,...,0,1,0,1,0,1,1,0,0,1
2,01000001001110110101011001,0011101000001001110110101011001,1,0100000000000000000000000000000,0111101000001001110110101011001,0,0,1,1,1,...,0,1,0,1,0,1,1,0,0,1
3,01000001001110110101011001,0011101000001001110110101011001,2,0010000000000000000000000000000,0001101000001001110110101011001,0,0,1,1,1,...,0,1,0,1,0,1,1,0,0,1
4,01000001001110110101011001,0011101000001001110110101011001,3,0001000000000000000000000000000,0010101000001001110110101011001,0,0,1,1,1,...,0,1,0,1,0,1,1,0,0,1


In [0]:
train_data, test_data, train_labels, test_labels = split_data(test_size=0.3)

In [0]:
# print(train_data)
# decoder = []
# with open('./dataset_files/h.txt') as f:
#     decoder = pd.read_csv(f, header=None, names=['h'], dtype=str) #, sep='\n')
# for j in range(len(decoder['h'][0])):
#     decoder['bin_' + str(j)] = decoder['h'][:].apply(lambda x: int(x[j]))
# decoder = decoder.drop(['h'], axis=1)
# decoder_np = decoder.as_matrix()
# print(decoder_np[:6])

In [0]:
# model = Sequential()
# model.add(Dense(31, input_shape=(31,), 
# #                 weights=decoder_np, 
#                 activation='relu', name='d1'))
# # model.add(Dense(100, activation='relu', name='d2'))
# # model.add(Dense(31, activation='softmax', name='d3'))
# model.compile(optimizer='rmsprop',
#               loss='categorical_crossentropy', # энтропия Шелдона / бинарная
#               metrics=['accuracy', 'mse'])

MCP = ModelCheckpoint('best_model.hd5', save_best_only=True)

In [0]:
# train_labels_categor = to_categorical(train_labels, num_classes=31)

model.fit(x=train_data, y=train_labels_categor, validation_split=0.2,  shuffle=True, epochs=100, callbacks=[MCP])

LSTM;

матринчая нейронная сеть, каждый слой - логические ячейки xor...
метрика хэмминга

подавать и правильные и неправильные


поставить керас
from keras... import
inp = Input(shape=(31,))
x = Dense(128, activation='elu')(inp)
x = Dense(128, activation='relu')(x)

test = Dense(.....)(x)

m = Dense(26, activation='пороговая функция типа step которой уже нет 'sigmoid')(x)

test = Dense(...)(test)
test= Dense(31....)(test)
m = Concatinate()([test,m])
          
model = Model(inp, m, 'm')
model.compile() , summary, fit

метки = бинарный код ошибки + информационное слово

лосс - бинарная кросс энтропия
оптимизатор - Адам
стартовая точноесть - 1е-5

In [0]:
# from keras.layers import Conv1D, Reshape, Flatten

# inp = Input(shape=(31,))
# x = Dense(128, activation='relu',  name='d1')(inp)
# x = Dense(128, activation='relu',  name='d2')(x)

# t = Dense(128, activation='elu', name='t1')(x)
# t = Dense(128, activation='relu', name='t2')(t)
# t = Dense(31, activation='softmax', name='t3')(t)

# m = Dense(26, activation='sigmoid',  name='d3')(x)
# m = Concatenate()([t, m])

# model = Model(inp, m, 'm')
# model.compile(loss='binary_crossentropy', optimizer='Adam')
# model.summary()

In [22]:
from keras.layers import Conv1D, Reshape, Flatten, MaxPooling1D, UpSampling1D
from keras.layers.recurrent import LSTM
from keras import models
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Concatenate, CuDNNLSTM
# from keras.utils import to_categorical
# from keras.callbacks import ModelCheckpoint

inp = Input(shape=(31,))
x = Dense(256, activation='tanh')(inp)
# x = Dense(1054, activation='relu')(x)
# x = Dense(64, activation='elu')(x)
x = Dense(512, activation='softsign')(x)
x = Dense(1024, activation='sigmoid')(x)
x = Dense(512, activation='softsign')(x)
x = Dense(256, activation='tanh')(x)
# x = Dense(1024, activation='relu')(x)
# x = Dense(1054, activation='relu')(x)

m = Dense(31, activation='sigmoid',  name='d3')(x)

model = Model(inp, m, 'm')
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 31)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               8192      
_________________________________________________________________
dense_7 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_8 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_9 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_10 (Dense)             (None, 256)               131328    
_________________________________________________________________
d3 (Dense)                   (None, 31)                7967      
Total para

In [23]:
model.fit(x=train_data, y=train_labels, validation_split=0.1,  shuffle=True, epochs=15, batch_size=32)#, callbacks=[MCP])tu

Train on 41287 samples, validate on 4588 samples
Epoch 1/15
41287/41287 [==============================] - 12s 299us/step - loss: 0.1878 - acc: 0.9392 - val_loss: 0.1292 - val_acc: 0.9658
Epoch 2/15
41287/41287 [==============================] - 12s 293us/step - loss: 0.0928 - acc: 0.9716 - val_loss: 0.0904 - val_acc: 0.9719
Epoch 3/15
 2272/41287 [>.............................] - ETA: 11s - loss: 0.0655 - acc: 0.9766

41287/41287 [==============================] - 12s 291us/step - loss: 0.0577 - acc: 0.9788 - val_loss: 0.0587 - val_acc: 0.9787
Epoch 4/15
41287/41287 [==============================] - 12s 292us/step - loss: 0.0336 - acc: 0.9868 - val_loss: 0.0431 - val_acc: 0.9831
Epoch 5/15
16960/41287 [===========>..................] - ETA: 6s - loss: 0.0198 - acc: 0.9925

41287/41287 [==============================] - 12s 294us/step - loss: 0.0185 - acc: 0.9929 - val_loss: 0.0313 - val_acc: 0.9874
Epoch 6/15
41287/41287 [==============================] - 12s 293us/step - loss: 0.0099 - acc: 0.9967 - val_loss: 0.0234 - val_acc: 0.9903
Epoch 7/15
20320/41287 [=============>................] - ETA: 5s - loss: 0.0051 - acc: 0.9988

41287/41287 [==============================] - 12s 293us/step - loss: 0.0050 - acc: 0.9987 - val_loss: 0.0199 - val_acc: 0.9922
Epoch 8/15
41287/41287 [==============================] - 12s 293us/step - loss: 0.0026 - acc: 0.9995 - val_loss: 0.0159 - val_acc: 0.9937
Epoch 9/15
21312/41287 [==============>...............] - ETA: 5s - loss: 0.0013 - acc: 0.9998

41287/41287 [==============================] - 12s 293us/step - loss: 0.0014 - acc: 0.9998 - val_loss: 0.0148 - val_acc: 0.9942
Epoch 10/15
41287/41287 [==============================] - 12s 294us/step - loss: 7.7975e-04 - acc: 0.9999 - val_loss: 0.0127 - val_acc: 0.9951
Epoch 11/15
20288/41287 [=============>................] - ETA: 6s - loss: 4.0250e-04 - acc: 1.0000

41287/41287 [==============================] - 12s 294us/step - loss: 4.3807e-04 - acc: 0.9999 - val_loss: 0.0130 - val_acc: 0.9953
Epoch 12/15
41287/41287 [==============================] - 12s 294us/step - loss: 2.7578e-04 - acc: 1.0000 - val_loss: 0.0106 - val_acc: 0.9960
Epoch 13/15
19200/41287 [============>.................] - ETA: 6s - loss: 1.4415e-04 - acc: 1.0000

41287/41287 [==============================] - 12s 293us/step - loss: 1.7456e-04 - acc: 1.0000 - val_loss: 0.0104 - val_acc: 0.9962
Epoch 14/15
41287/41287 [==============================] - 12s 293us/step - loss: 1.0559e-04 - acc: 1.0000 - val_loss: 0.0090 - val_acc: 0.9967
Epoch 15/15
19008/41287 [============>.................] - ETA: 6s - loss: 4.6797e-05 - acc: 1.0000

41287/41287 [==============================] - 12s 295us/step - loss: 6.4495e-05 - acc: 1.0000 - val_loss: 0.0082 - val_acc: 0.9971


### Проверим, сколько слов исправлено точно

In [0]:
y_pred = model.predict(test_data)

In [25]:
print(test_data.shape[0])
test_data.shape[0] / 32

19661


614.40625

In [0]:
print(y_pred.shape)
# print(test_labels[0], test_data[0])

In [0]:
def probs_to_labels(predicted_probs):
    return [1 if x > 0.5 else 0 for x in predicted_probs]
def count_errors(y, y_pred):
  count = 0
  for i in range (0,31):
    labelBit = y[i]
    resultBit = y_pred[i]
    if labelBit != resultBit:
      count += 1
  return count


In [26]:
errorStats = {'0': 0}
for i in range(0, y_pred.shape[0]):
  resultArray = probs_to_labels(y_pred[i])
  errorNum = count_errors(test_labels[i], resultArray)
  if errorStats.get(str(errorNum)) == None:
    errorStats[str(errorNum)] = 0
  errorStats[str(errorNum)] += 1
print(errorStats)

{'0': 58166, '1': 512901, '2': 55074, '3': 2893, '4': 105, '5': 7}


In [26]:
def binary_accuracy(y, y_pred):
    return sum(1 if np.array_equal(a, probs_to_labels(b)) else 0 for (a,b) in zip(y, y_pred)) / y.shape[0]
print(np.array(probs_to_labels(y_pred[0])))
print(test_labels[0])

[0 0 0 0 0 1 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 1 0]
[0 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 1 0]


In [0]:
accuracy = binary_accuracy(test_labels, y_pred)

### optimizers scores:
- Adam - 0.3 accuracy
- SGD - 0.05
- RMSprop - 0.29

In [34]:
2 ** 21 * 0.3

629145.6

0. Сделать датасет поменьше
1. Поменять кодирующую матрицу Хэмминга (генерирующую - g.txt) -> Попробовать потренировать
2. Прочитать статью Бернштейна и оттуда перенять архитектуру
